In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score, ShuffleSplit
import numpy as np
from itertools import combinations
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import time
import matplotlib.pyplot as plt
%matplotlib inline

c:\users\tim\anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def import_data(experimental_file, response_file, response):
    'imports data'
    'experimental_file = experimental design csv filename i.e experimental.csv'
    'response_file = results csv filename i.e Response.csv'
    'response = reponse name: i.e rheomix final deg time min or rheomix stability time min'
    
    experimental_df = pd.read_csv(experimental_file)
    response_df = pd.read_csv(response_file)
    
    X = experimental_df[experimental_df.columns.values.tolist()[1:]].values
    y = response_df[response].values
    max1 = max(y)
    min1 = min(y)

    y_norm = [2*((i-min1)/(max1-min1)) - 1 for i in y]
    
    
    X_linear = X
    linear_terms = experimental_df.columns.values.tolist()[1:]
    
    return y_norm, X_linear, linear_terms, experimental_df, response_df
    

In [3]:
def X_gen(model, X_linear):

    for i, j in enumerate(model):
        
        if i == 0 and len(j) == 2: 
            X_new = X_linear[:, model[0][1]]
        
        else:
            
            if len(j) == 2:
                add = X_linear[:, j[1]]

            if len(j) == 3:
                add = X_linear[:, j[1]]*X_linear[:, j[2]]

            X_new2 = np.column_stack((X_new, add))
            X_new = X_new2

    
    return X_new

In [4]:
start = time.time()


# linear_terms = experimental_df.columns.values.tolist()[1:]



def fit_model(experimental_file, response_file, response, AIC_lim, CN_lim):
    

    total_subset = []
    model_subset = []
    
    y, X_linear, linear_terms, experimental_df, response_df = import_data(experimental_file, response_file, response)
    y1 = np.array(y)
    lin_terms = []
    AIC_prev = 1000

    for i in range(len(linear_terms)):
        term = linear_terms[i]
        key = i
        lin_terms.append([term, i])

    cnt = 0
    for i in range(len(lin_terms)):
        for j in combinations(lin_terms, i+1):
            linear_terms = list(j)


            model = [a for a in linear_terms]
            cnt+=1

            X = X_gen(model, X_linear)

            model_fit = sm.OLS(y, X)
            results = model_fit.fit()
            AIC_cur = results.aic
            
            if len(model) == 1:
                
                X1 = X.reshape(len(X), 1)
                model_obj = LinearRegression(fit_intercept=False)
                my_cv = ShuffleSplit(len(y1), n_iter=3, test_size=0.333, random_state=0)

                score = cross_val_score(model_obj, X1, y1, cv=my_cv)                
            
            else:
            
                model_obj = LinearRegression(fit_intercept=False)
                my_cv = ShuffleSplit(len(y1), n_iter=3, test_size=0.333, random_state=0)

                score = cross_val_score(model_obj, X, y1, cv=my_cv)

            print(score, model)
            
            model_name = '--'
            for i in model:
                
                model_name += i[0] + "--"

            dictionary = {'AIC': results.aic, 'Cond_No': results.condition_number, 'Model': model_name,
                          'r2': results.rsquared, 'No_terms': len(model), 'Kfold': score}
            total_subset.append(dictionary)
            
            if AIC_cur < AIC_lim:
                if results.condition_number < CN_lim:
                    
                    variables = model_fit.exog
                    
                    if len(model) != 1:

                        vif = [variance_inflation_factor(variables, i) for i in range(variables.shape[1])]
                        vif_max = max(vif)

                    else:
                        vif_max = 0


                    if max(vif) < 40:
                        dictionary = {'AIC': results.aic, 'Cond_No': results.condition_number, 'Model': model,
                        'r2': results.rsquared, 'No_terms': len(model), 'VIF': vif_max, 'Kfold': score}
                        
                        model_subset.append(dictionary)

                    
                    
            if AIC_cur < AIC_prev:

                AIC_prev = AIC_cur
                final_model = [model, results, AIC_cur]
                




            poss_terms = []
            for i in range(len(linear_terms)):
                for j in range(len(linear_terms)): 
                    if i < j:
                        poss_terms.append([linear_terms[i][0] + '*' + linear_terms[j][0],  linear_terms[i][1], linear_terms[j][1]])



            for m in range(1, len(poss_terms) + 1):
                for k in combinations(poss_terms, m):

                    model = [a for a in linear_terms]
                    for i in range(m):
                        model.append(k[i])

                    cnt +=1
                    X = X_gen(model, X_linear)

                    model_fit = sm.OLS(y, X)
                    results = model_fit.fit()
                    AIC_cur = results.aic
                    
                    model_obj = LinearRegression(fit_intercept=False)
                    my_cv = ShuffleSplit(len(y1), n_iter=3, test_size=0.333, random_state=0)
                    
                    score = cross_val_score(model_obj, X, y1, cv=my_cv)
                    print(score, model)
                    
                    
                    model_name = '--'
                    for i in model:

                        model_name += i[0] + "--"
                    
                    dictionary = {'AIC': results.aic, 'Cond_No': results.condition_number, 'Model': model_name,
                                  'r2': results.rsquared, 'No_terms': len(model), 'Kfold': score}
                    
                    total_subset.append(dictionary)



                    if AIC_cur < AIC_lim:
                        if results.condition_number < CN_lim:

                            variables = model_fit.exog
                            
                            if len(model) != 1:

                                vif = [variance_inflation_factor(variables, i) for i in range(variables.shape[1])]
                                vif_max = max(vif)

                            else:
                                vif_max = 0


                            if max(vif) < 40:
                                dictionary = {'AIC': results.aic, 'Cond_No': results.condition_number, 'Model': model,
                                'r2': results.rsquared, 'No_terms': len(model), 'VIF': vif_max, 'Kfold': score}
                                
                                model_subset.append(dictionary)


                    if AIC_cur < AIC_prev:
                        
                        AIC_prev = AIC_cur
                        final_model = [model, results, AIC_cur]




    return final_model, total_subset, model_subset


In [5]:
experimental_file = 'experimental.csv'
response_file = 'Response.csv'


test1 = 'rheomix final deg time min'
test2 = 'rheomix stability time min'
response = test1

AIC_lim = 20
CN_lim = 600


final_model, total_subset, model_subset = fit_model(experimental_file, response_file, response, AIC_lim, CN_lim)

[-0.07518123  0.04066424  0.00595714] [['Xpvc', 0]]
[-0.05665273 -0.24092534 -0.14585932] [['Xfiller', 1]]
[-0.57515999 -0.71887221 -0.40171308] [['Xfr', 2]]
[-0.37080128 -0.12605969 -0.21361922] [['Xstabiliser', 3]]
[-0.67173981 -0.70208221 -0.65447151] [['Xdinp', 4]]
[-0.84411323 -1.01356148 -1.12292237] [['Xldh', 5]]
[-1.00014419 -0.59635131 -0.37607724] [['Xsph', 6]]
[0.05828399 0.12315815 0.10136944] [['Xpvc', 0], ['Xfiller', 1]]
[0.05673939 0.16271353 0.14270511] [['Xpvc', 0], ['Xfiller', 1], ['Xpvc*Xfiller', 0, 1]]
[-0.07542445 -0.01784748  0.00319582] [['Xpvc', 0], ['Xfr', 2]]
[-0.12279031 -0.04051106 -0.02182254] [['Xpvc', 0], ['Xfr', 2], ['Xpvc*Xfr', 0, 2]]
[-0.07560192 -0.00051629  0.00469958] [['Xpvc', 0], ['Xstabiliser', 3]]
[-0.07719342 -0.01871565 -0.00331712] [['Xpvc', 0], ['Xstabiliser', 3], ['Xpvc*Xstabiliser', 0, 3]]
[0.11221871 0.21279414 0.15508477] [['Xpvc', 0], ['Xdinp', 4]]
[0.15797592 0.22854112 0.15961258] [['Xpvc', 0], ['Xdinp', 4], ['Xpvc*Xdinp', 0, 4]]
[0.0

[0.03406847 0.10149564 0.05499371] [['Xpvc', 0], ['Xfr', 2], ['Xldh', 5], ['Xfr*Xldh', 2, 5]]
[-2.05833913  0.17512119  0.17191867] [['Xpvc', 0], ['Xfr', 2], ['Xldh', 5], ['Xpvc*Xfr', 0, 2], ['Xpvc*Xldh', 0, 5]]
[0.01623157 0.07722659 0.04458542] [['Xpvc', 0], ['Xfr', 2], ['Xldh', 5], ['Xpvc*Xfr', 0, 2], ['Xfr*Xldh', 2, 5]]
[-1.46757914  0.10690172  0.08487215] [['Xpvc', 0], ['Xfr', 2], ['Xldh', 5], ['Xpvc*Xldh', 0, 5], ['Xfr*Xldh', 2, 5]]
[-2.04536587  0.16384774  0.13503573] [['Xpvc', 0], ['Xfr', 2], ['Xldh', 5], ['Xpvc*Xfr', 0, 2], ['Xpvc*Xldh', 0, 5], ['Xfr*Xldh', 2, 5]]
[-0.06430677 -0.05085534  0.00489202] [['Xpvc', 0], ['Xfr', 2], ['Xsph', 6]]
[-0.11028487 -0.05239251 -0.00983947] [['Xpvc', 0], ['Xfr', 2], ['Xsph', 6], ['Xpvc*Xfr', 0, 2]]
[-0.03350962 -0.03642476  0.01501062] [['Xpvc', 0], ['Xfr', 2], ['Xsph', 6], ['Xpvc*Xsph', 0, 6]]
[-0.17843133 -0.04996987 -0.01693282] [['Xpvc', 0], ['Xfr', 2], ['Xsph', 6], ['Xfr*Xsph', 2, 6]]
[-0.09473024 -0.04218762 -0.02329154] [['Xpvc', 0

[-0.02662604 -0.09265161 -0.15908847] [['Xfiller', 1], ['Xfr', 2], ['Xldh', 5], ['Xfr*Xldh', 2, 5]]
[ 0.12115711 -0.29983398 -0.433819  ] [['Xfiller', 1], ['Xfr', 2], ['Xldh', 5], ['Xfiller*Xfr', 1, 2], ['Xfiller*Xldh', 1, 5]]
[ 0.03017694 -0.20512803 -0.37101643] [['Xfiller', 1], ['Xfr', 2], ['Xldh', 5], ['Xfiller*Xfr', 1, 2], ['Xfr*Xldh', 2, 5]]
[ 0.0426643  -0.15398411 -0.21404074] [['Xfiller', 1], ['Xfr', 2], ['Xldh', 5], ['Xfiller*Xldh', 1, 5], ['Xfr*Xldh', 2, 5]]
[ 0.0544822  -0.29026079 -0.45448009] [['Xfiller', 1], ['Xfr', 2], ['Xldh', 5], ['Xfiller*Xfr', 1, 2], ['Xfiller*Xldh', 1, 5], ['Xfr*Xldh', 2, 5]]
[-0.16126685 -0.12195139  0.03500095] [['Xfiller', 1], ['Xfr', 2], ['Xsph', 6]]
[-0.10589242 -0.19103337 -0.09459369] [['Xfiller', 1], ['Xfr', 2], ['Xsph', 6], ['Xfiller*Xfr', 1, 2]]
[-0.12042229 -0.20211253  0.01213953] [['Xfiller', 1], ['Xfr', 2], ['Xsph', 6], ['Xfiller*Xsph', 1, 6]]
[-0.24632824 -0.05677855  0.10403949] [['Xfiller', 1], ['Xfr', 2], ['Xsph', 6], ['Xfr*Xsph',

[0.08497852 0.4559171  0.39232849] [['Xfr', 2], ['Xdinp', 4], ['Xldh', 5], ['Xfr*Xldh', 2, 5], ['Xdinp*Xldh', 4, 5]]
[0.0415074  0.44454366 0.39250191] [['Xfr', 2], ['Xdinp', 4], ['Xldh', 5], ['Xfr*Xdinp', 2, 4], ['Xfr*Xldh', 2, 5], ['Xdinp*Xldh', 4, 5]]
[-0.53293333 -0.46824398 -0.29299498] [['Xfr', 2], ['Xdinp', 4], ['Xsph', 6]]
[-0.33963186 -0.38338872 -0.10883056] [['Xfr', 2], ['Xdinp', 4], ['Xsph', 6], ['Xfr*Xdinp', 2, 4]]
[-0.52582137 -0.28201333 -0.09263493] [['Xfr', 2], ['Xdinp', 4], ['Xsph', 6], ['Xfr*Xsph', 2, 6]]
[ 0.0180376  -0.07216412 -0.06104829] [['Xfr', 2], ['Xdinp', 4], ['Xsph', 6], ['Xdinp*Xsph', 4, 6]]
[-0.37953075 -0.27798103 -0.00257224] [['Xfr', 2], ['Xdinp', 4], ['Xsph', 6], ['Xfr*Xdinp', 2, 4], ['Xfr*Xsph', 2, 6]]
[-0.06343602 -0.16967505 -0.10420664] [['Xfr', 2], ['Xdinp', 4], ['Xsph', 6], ['Xfr*Xdinp', 2, 4], ['Xdinp*Xsph', 4, 6]]
[-0.13144716  0.01137895 -0.07264756] [['Xfr', 2], ['Xdinp', 4], ['Xsph', 6], ['Xfr*Xsph', 2, 6], ['Xdinp*Xsph', 4, 6]]
[-0.204654

[ 0.08940974 -0.0357409   0.08145631] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xpvc*Xfiller', 0, 1], ['Xpvc*Xstabiliser', 0, 3], ['Xfiller*Xstabiliser', 1, 3]]
[0.18977465 0.19808148 0.22144125] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xpvc*Xfiller', 0, 1], ['Xpvc*Xstabiliser', 0, 3], ['Xfr*Xstabiliser', 2, 3]]
[ 0.02852771 -0.00138694  0.02535877] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xpvc*Xfiller', 0, 1], ['Xfiller*Xfr', 1, 2], ['Xfiller*Xstabiliser', 1, 3]]
[0.15910148 0.18549132 0.12741751] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xpvc*Xfiller', 0, 1], ['Xfiller*Xfr', 1, 2], ['Xfr*Xstabiliser', 2, 3]]
[0.18923603 0.15856514 0.14469219] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xpvc*Xfiller', 0, 1], ['Xfiller*Xstabiliser', 1, 3], ['Xfr*Xstabiliser', 2, 3]]
[-0.13467697 -0.2092839  -0.02703707] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xpvc*Xfr', 

...

[-0.60373621  0.37611112  0.43246361] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xldh', 5], ['Xpvc*Xfiller', 0, 1], ['Xpvc*Xstabiliser', 0, 3], ['Xfr*Xldh', 2, 5], ['Xstabiliser*Xldh', 3, 5]]
[-3.53504914  0.31733152  0.21621481] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xldh', 5], ['Xpvc*Xfiller', 0, 1], ['Xpvc*Xldh', 0, 5], ['Xfiller*Xfr', 1, 2], ['Xfiller*Xstabiliser', 1, 3]]
[-2.84706215  0.419885    0.20342789] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xldh', 5], ['Xpvc*Xfiller', 0, 1], ['Xpvc*Xldh', 0, 5], ['Xfiller*Xfr', 1, 2], ['Xfiller*Xldh', 1, 5]]
[-4.11126373  0.40561955  0.25834631] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xldh', 5], ['Xpvc*Xfiller', 0, 1], ['Xpvc*Xldh', 0, 5], ['Xfiller*Xfr', 1, 2], ['Xfr*Xstabiliser', 2, 3]]
[-3.57422057  0.32759901  0.20077776] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xldh', 5], ['Xpvc*Xfiller', 0, 1], ['Xpvc*Xldh', 0, 5], ['Xf

[-0.33267915  0.39730459  0.45580985] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xldh', 5], ['Xpvc*Xfr', 0, 2], ['Xpvc*Xstabiliser', 0, 3], ['Xfiller*Xstabiliser', 1, 3], ['Xstabiliser*Xldh', 3, 5]]
[-0.03148116  0.19999408  0.1434829 ] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xldh', 5], ['Xpvc*Xfr', 0, 2], ['Xpvc*Xstabiliser', 0, 3], ['Xfiller*Xldh', 1, 5], ['Xfr*Xstabiliser', 2, 3]]
[-0.07467019  0.14222399  0.08313437] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xldh', 5], ['Xpvc*Xfr', 0, 2], ['Xpvc*Xstabiliser', 0, 3], ['Xfiller*Xldh', 1, 5], ['Xfr*Xldh', 2, 5]]
[-0.31935262  0.36550353  0.51382844] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xldh', 5], ['Xpvc*Xfr', 0, 2], ['Xpvc*Xstabiliser', 0, 3], ['Xfiller*Xldh', 1, 5], ['Xstabiliser*Xldh', 3, 5]]
[0.05504482 0.19681719 0.17077159] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xldh', 5], ['Xpvc*Xfr', 0, 2], ['Xpvc*Xstabiliser',

[-0.8619594   0.59159054  0.73443052] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xldh', 5], ['Xpvc*Xstabiliser', 0, 3], ['Xpvc*Xldh', 0, 5], ['Xfiller*Xldh', 1, 5], ['Xstabiliser*Xldh', 3, 5]]
[-4.15169093  0.41890128  0.42223144] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xldh', 5], ['Xpvc*Xstabiliser', 0, 3], ['Xpvc*Xldh', 0, 5], ['Xfr*Xstabiliser', 2, 3], ['Xfr*Xldh', 2, 5]]
[-2.43667035  0.57669549  0.62897201] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xldh', 5], ['Xpvc*Xstabiliser', 0, 3], ['Xpvc*Xldh', 0, 5], ['Xfr*Xstabiliser', 2, 3], ['Xstabiliser*Xldh', 3, 5]]
[-2.17517066  0.4772351   0.52052168] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xldh', 5], ['Xpvc*Xstabiliser', 0, 3], ['Xpvc*Xldh', 0, 5], ['Xfr*Xldh', 2, 5], ['Xstabiliser*Xldh', 3, 5]]
[-0.11408605  0.12542083 -0.1834482 ] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xldh', 5], ['Xpvc*Xstabiliser', 0, 3], ['Xfiller

[-0.22227818  0.17878521  0.25103712] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xldh', 5], ['Xfiller*Xfr', 1, 2], ['Xfiller*Xstabiliser', 1, 3], ['Xfr*Xldh', 2, 5], ['Xstabiliser*Xldh', 3, 5]]
[ 0.07323855  0.18709693 -0.09094712] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xldh', 5], ['Xfiller*Xfr', 1, 2], ['Xfiller*Xldh', 1, 5], ['Xfr*Xstabiliser', 2, 3], ['Xfr*Xldh', 2, 5]]
[-0.22770886  0.36937021  0.48745462] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xldh', 5], ['Xfiller*Xfr', 1, 2], ['Xfiller*Xldh', 1, 5], ['Xfr*Xstabiliser', 2, 3], ['Xstabiliser*Xldh', 3, 5]]
[-0.39882197  0.2347402   0.32561368] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xldh', 5], ['Xfiller*Xfr', 1, 2], ['Xfiller*Xldh', 1, 5], ['Xfr*Xldh', 2, 5], ['Xstabiliser*Xldh', 3, 5]]
[-0.11953745  0.26233175  0.37349977] [['Xpvc', 0], ['Xfiller', 1], ['Xfr', 2], ['Xstabiliser', 3], ['Xldh', 5], ['Xfiller*Xfr', 1, 2], ['Xfr*Xstabiliser',

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(total_subset)
df.to_csv("Toatal_model_subset_"  + response + ".csv")

dff = pd.DataFrame(model_subset)
dff.to_csv("limited_model_subset_"  + response + ".csv")

In [ ]:
# plt.plot(CN_hist, AIC_hist, 'r*')
# plt.axis([0, 2000, 12, 20])
# plt.xlabel('CN')
# plt.ylabel('AIC')
# plt.show()

In [ ]:
# plt.plot(VIF_max_hist, AIC_hist1, 'r*')
# plt.show()

In [ ]:
# plt.scatter(CN_hist1, AIC_hist1, c = VIF_max_hist, vmin=0, vmax=50)
# # plt.axis([0, 1000, 12, 20])
# plt.xlabel('CN')
# plt.ylabel('AIC')
# plt.colorbar()
# # plt.set_label('VIF')
# plt.show()

In [ ]:
# from mpl_toolkits.mplot3d import Axes3D

# fig = plt.figure()
# ax = Axes3D(fig)

# ax.scatter(CN_hist1,VIF_max_hist, AIC_hist1)
# plt.xlabel('CN')
# plt.ylabel('VIF')
# ax.set_zlabel('AIC')
# # plt.axis([0, 1000, 12, 18])
# # plt.colorbar()
# plt.show()

In [ ]:
# len(model_subset)

In [ ]:
# for i in model_subset:
#     if i[2] < 550 and i[2] > 450:
#         a = []
#         for j in i[0]:
#             a.append(j[0])
#         print('----model:', a, 'AIC', round(i[1], 3), 'CN:', round(i[2], 3), 'vif_max:', round(i[3], 3), 'R2:', round(i[4], 3), '----'
#              )
        

# 